In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import statsmodels.api as sm
from pandas import ExcelWriter
import xlsxwriter

In [2]:
# stationary or not, change rho and initial
# correlation
# constraint or not
# sample size T
# replications

In [3]:
T = 1000

In [4]:
replications = 100

In [5]:
theta_true = [0.3, 0.6]

In [6]:
theta = theta[0]

NameError: name 'theta' is not defined

In [ ]:
theta[0]

In [ ]:
theta = [[0.3,0.6], [0.8,0.6]]

In [ ]:
u = [theta[0] * i + theta[1] * j for (i,j) in zip(x1,x2)]

In [7]:
gamma_to_choose = [1,2,3]

In [8]:
initial = 0

In [9]:
T/10

100.0

In [10]:
def xy_simulation(theta, gamma, T, fun, stationary, error_type, constrain):
    
    # coefficients to construct x1, x2
    if stationary == 'stationary':
        rho = [0.5, 0.5]
        initial = 0
    else:
        initial = 100
        rho = [1, 1]
    
    if constrain == 'constrain':
        theta = theta[1]
    else:
        theta = theta[0]
    
    # error terms for x1, x2
    if error_type == 'standard_normal':
        v1 = np.random.standard_normal(T + initial)
        v2 = np.random.standard_normal(T + initial)
    else:
        pass
    

    # construct x1, x2
    x1 = [0]
    x2 = [0]
    i = 0
    while i < T + initial:
        x1.append(rho[0] * x1[i] + v1[i])
        x2.append(rho[1] * x2[i] + v2[i])
        i += 1
    
    if stationary == 'stationary':
        x1 = x1[1:]
        x2 = x2[1:]
        
    else:
        x1 = x1[initial+1:]
        x2 = x2[initial+1:]
        
    # construct single index
    u = [theta[0] * i + theta[1] * j for (i,j) in zip(x1,x2)]
    
    # construct dependent variable: y
    if fun == 'Polynomial':
        y = [gamma[0] + gamma[1] * i + gamma[2] * i**2 for i in u]
    elif fun =='sin':
        y = [np.sin(gamma[0] + i) for i in u]
    elif fun =='cos':
        y = [np.cos(gamma[0] + i) for i in u]
    elif fun =='exp_shift':
        y = [np.exp((-gamma[1])*(i-gamma[0])**2) for i in u]
    elif fun =='exp':
        y = [gamma[0] * np.exp(-i**2) for i in u]
    
    # combine results for return
    simulated_dataset = pd.DataFrame({'y' : y, 'x1' : x1, 'x2' : x2 })
    
    return simulated_dataset
#     return u

In [11]:
# xy_simulation(theta = theta, gamma = gamma_to_choose, T =100, fun = 'Polynomial', 
#               stationary = 'stationary', error_type = 'standard_normal', constrain = 'constrain')

In [38]:
def constraint(x):
    con = 0
    for j in np.arange(0, paper.shape[1]):
        con += x[j+1]**2
        cons = con - 1
    return cons
cons = {'type':'eq', 'fun': constraint}

In [39]:
def my_nls(fun, T, constrain, paper, y):
    # Initial Values:
    if fun == 'Polynomial':
        x0 = [0.001] * (paper.shape[1] + 3)
    elif fun == 'exp_shift':
        x0 = [0.001] * (paper.shape[1]+2)
    else: 
        x0 = [0.001] * (paper.shape[1]+1)
            
    Mn = [1 if np.sqrt(paper.iloc[i]['x1']**2 + paper.iloc[i]['x2']**2) < np.sqrt(T) else 0 for i in np.arange(0, T)]
    
    def nls(x, paper = paper, y = y, constrain = constrain):
        u = 0
        sum = 0
        if constrain =='constrain':
            if fun == 'Polynomial':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += ((y[i+1] - x[0] - x[j+2] * u[i] - x[j+3] * (u[i]**2))**2)*Mn[i]
            elif fun == 'Polynomial_3':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += ((y[i+1] - x[0] - x[j+2] * u[i] - x[j+3] * (u[i]**2) - x[j+4] * (u[i]**3))**2)*Mn[i]
            elif fun == 'Polynomial_4':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += ((y[i+1] - x[0] - x[j+2] * u[i] - x[j+3] * (u[i]**2) - x[j+4] * (u[i]**3) - x[j+5] * (u[i]**4))**2)*Mn[i]
            elif fun == 'sin':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += ((y[i+1] - np.sin(u[i]+x[0]))**2)*Mn[i]
            elif fun == 'cos':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += ((y[i+1] - np.cos(u[i]+x[0]))**2)*Mn[i]    
            elif fun == 'exp_shift':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += ((y[i+1] - np.exp((-x[j+2])*(u[i]-x[0])**2))**2)*Mn[i]
#                 print(sum)
            elif fun == 'exp':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += ((y[i+1] - x[0] * np.exp(-(u[i]**2)))**2)*Mn[i]
        else:
            if fun == 'Polynomial':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += (y[i+1] - x[0] - x[j+2] * u[i] - x[j+3] * (u[i]**2))**2
            elif fun == 'Polynomial_3':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += (y[i+1] - x[0] - x[j+2] * u[i] - x[j+3] * (u[i]**2) - x[j+4] * (u[i]**3))**2
            elif fun == 'Polynomial_4':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += (y[i+1] - x[0] - x[j+2] * u[i] - x[j+3] * (u[i]**2) - x[j+4] * (u[i]**3) - x[j+5] * (u[i]**4))**2
            elif fun == 'sin':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += (y[i+1] - np.sin(u[i]+x[0]))**2
            elif fun == 'cos':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += (y[i+1] - np.cos(u[i]+x[0]))**2    
            elif fun == 'exp_shift':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += (y[i+1] - np.exp((-x[j+2])*(u[i]-x[0])**2))**2
            elif fun == 'exp':
                for j in np.arange(0, paper.shape[1]):
                    u += paper.iloc[:,j] * x[j+1]
                for i in np.arange(0, len(y)-1):
                    sum += (y[i+1] - x[0] * np.exp(-(u[i]**2)))**2
        return sum
        
    if constrain == 'constrain':
        return minimize(nls, x0, method = 'SLSQP', constraints = cons)
    else:
        return minimize(nls, x0, method = 'L-BFGS-B')
#         if fun == 'exp_shift':
#             return minimize(nls, x0, method = 'L-BFGS-B')
#         else:
#             return minimize(nls, x0, method = 'SLSQP')



In [55]:
def simulation(theta, gamma_, replications,  T, fun, stationary, error_type, constrain):
    
    if fun == 'Polynomial':
        gamma = gamma_to_choose
    elif fun =='exp_shift':
        gamma = gamma_to_choose[0:2]
    else:
        gamma = [gamma_to_choose[0]]
        
    if constrain == 'constrain':
        theta_true = theta[1]
    else:
        theta_true = theta[0]
    print(T)
    
    i = 1
    theta_est = np.array([]).reshape(-1,2)
    gamma_est = np.array([]).reshape(-1,len(gamma))
    while i <= replications:
        simulated_dataset = xy_simulation(theta = theta, gamma = gamma, T = T, fun = fun, 
                                          stationary = stationary, error_type = error_type, constrain = constrain)
        paper = simulated_dataset[['x1','x2']]
#         def constraint(x):
#             con = 0
#             for j in np.arange(0, paper.shape[1]):
#                 con += x[j+1]**2
#                 cons = con - 1
#             return cons
        cons = {'type':'eq', 'fun': constraint}
        result = my_nls(fun = fun, T = T, constrain = constrain, paper = paper, y = simulated_dataset['y'])
        print(result.x)
        theta_est = np.vstack([theta_est, result.x[1:3]])
        gamma1 = np.delete(result.x, [1,2])
        gamma_est = np.vstack([gamma_est, gamma1])
        i = i+1
        
    # Reformat the results
    bias_theta, std_theta, rmse_theta = np.nanmean(theta_est - theta_true, axis = 0), np.nanstd(theta_est - theta_true, axis = 0), np.sqrt(np.nanmean((theta_est - theta_true)**2, axis = 0))
    bias_gamma, std_gamma, rmse_gamma = np.nanmean(gamma_est - gamma, axis = 0), np.nanstd(gamma_est - gamma, axis = 0), np.sqrt(np.nanmean((gamma_est - gamma)**2, axis = 0))
    
    # Add hierarchical index
    iterables = [['T = ' + str(T)], ['True_values', 'Bias', 'std', 'RMSE']]
    
    theta_result = pd.DataFrame([theta_true, bias_theta, std_theta, rmse_theta], 
                                index = pd.MultiIndex.from_product(iterables, names=['Sample Size', 'Values']), 
                                columns = ['theta_1', 'theta_2'])
    
    
    gamma_result = pd.DataFrame([gamma, bias_gamma, std_gamma, rmse_gamma], 
                                index = theta_result.index, 
                                columns = ['gamma_' + str(i) for i in np.arange(1, len(gamma)+1)])
    
    # Joint theta and gamma results
    result_combine = theta_result.merge(gamma_result, left_index=True, right_on = ['Sample Size', 'Values'])
    
    # Add hierarchical header
    header = pd.MultiIndex.from_product([[stationary + '_' + constrain], 
                                         [result_combine.columns[i] for i in np.arange(0,len(theta)+len(gamma))]])
    result_combine.columns = header

    return result_combine
#     return len(simulated_dataset)

In [56]:
# Initilazation
T_list = [100, 500, 1000]
sta_list = ['stationary', 'nonstationary']
con_list = ['constrain', 'no-constrain']
# fun_list = ['Polynomial', 'sin', 'cos', 'exp_shift', 'exp']
# fun_list = ['exp_shift']
replications = 10
theta = [[0.3,0.6], [0.8,0.6]]
gamma_to_choose = [0.2,0.5,0.8]

In [53]:
# simulated_dataset = xy_simulation(theta = theta, gamma = gamma_to_choose, T = T, fun = 'sin', 
#                                   stationary = 'stationary', error_type = 'standard_normal', constrain = 'constrain')
# simulated_dataset.head()

In [49]:
paper = simulated_dataset[['x1','x2']]
my_nls(fun = 'Polynomial', T = 100, constrain = 'nconstrain', paper = paper, y = simulated_dataset['y'])

      fun: 360.17001265138754
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 2.85353963e-03, -1.47792889e-04, -2.67164069e-04,  7.38964445e-05,
        1.42108547e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 120
      nit: 9
   status: 0
  success: True
        x: array([ 0.10377452,  0.38232634,  0.28100666,  0.54933623, -0.06816884])

In [57]:
simulation(theta = theta , gamma_ = gamma_to_choose, replications = 5, T = 50, fun = 'Polynomial', stationary = 'stationary', 
                            error_type = 'standard_normal', constrain = 'nconstrain')

50
[ 0.88667082  0.42904037  0.19802959  0.15738274 -0.66788184]
[0.76697459 0.5171678  0.21884428 0.58093805 0.24101818]
[0.42157883 0.19091875 0.51893539 0.47863203 0.6273387 ]
[ 0.36819837 -0.33460947  0.74166818  0.21718304  0.75408904]
[ 0.37641969  0.45982102 -0.08338423  0.13859727  0.41671751]


stationary_nconstrain                                \
                                      theta_1   theta_2   gamma_1   gamma_2   
Sample Size Values                                                            
T = 50      True_values              0.300000  0.600000  0.200000  0.500000   
            Bias                    -0.047532 -0.281181  0.363968 -0.185453   
            std                      0.313907  0.284675  0.218689  0.180570   
            RMSE                     0.317485  0.400129  0.424615  0.258840   

                                   
                          gamma_3  
Sample Size Values                 
T = 50      True_values  0.800000  
            Bias        -0.525744  
            std          0.502778  
            RMSE         0.727456

In [ ]:
appended_data = []
writer = pd.ExcelWriter('Polynomial.xlsx', engine='xlsxwriter')
for j in sta_list:
    print(j)
    for i in con_list:
        print(i)
        frame = [simulation(theta = theta , gamma_ = gamma_to_choose, replications = replications, T = T, fun = 'Polynomial', stationary = j, 
                            error_type = 'standard_normal', constrain = i) for T in T_list]
        sub_results = pd.concat(frame)  
        appended_data.append(sub_results)
    result = pd.concat(appended_data, axis = 1)
    result.to_excel(writer, sheet_name=sheetname, index = True)
writer.save()

In [1548]:
# %%time
# writer = pd.ExcelWriter('Polynomial.xlsx', engine='xlsxwriter')
# for k in fun_list:
#     print('start                                  ' + ':                              ' + k)
#     appended_data = []
#     sheetname = k
#     for j in sta_list:
#         initial = T/10
#         print(j)
#         for i in con_list:
#             print(i)
#             frame = [simulation(theta , gamma , replications = replications, T = T, fun = k, stationary = j, 
#                                 error_type = 'standard_normal', constrain = i) for T in T_list]
#             sub_results = pd.concat(frame)  
#             appended_data.append(sub_results)
#         result = pd.concat(appended_data, axis = 1)
#         result.to_excel(writer, sheet_name=sheetname, index = True)
# writer.save()

start                                  :                              Polynomial
stationary
constrain
100


TypeError: 'float' object is not subscriptable

In [1365]:
result

stationary_constrain                      \
                                     theta_1   theta_2   gamma_1   
Sample Size Values                                                 
T = 100     True_values             0.800000  0.600000  0.200000   
            Bias                   -0.835668 -0.189558 -0.021076   
            std                     0.565120  0.714776  0.012421   
            RMSE                    1.008812  0.739485  0.024464   
T = 500     True_values             0.800000  0.600000  0.200000   
            Bias                   -0.694630  0.172920 -0.016318   
            std                     0.591959  0.202671  0.006020   
            RMSE                    0.912648  0.266415  0.017393   

                        stationary_no-constrain                      \
                                        theta_1   theta_2   gamma_1   
Sample Size Values                                                    
T = 100     True_values                0.300000  0.600000  0.200000   
            Bias                      -0.227421 -0.419360 -0.057892   
            std                        0.094047  0.128236  0.008930   
            RMSE                       0.246099  0.438528  0.058577   
T = 500     True_values                0.300000  0.600000  0.200000   
            Bias                      -0.205711 -0.363814 -0.054328   
            std                        0.016090  0.024846  0.002225   
            RMSE                       0.206339  0.364662  0.054373   

                        nonstationary_constrain                      \
                                        theta_1   theta_2   gamma_1   
Sample Size Values                                                    
T = 100     True_values                0.800000  0.600000  0.200000   
            Bias                      -0.103166  0.106739 -0.051468   
            std                        0.090605  0.082052  0.385173   
            RMSE                       0.137304  0.134632  0.388597   
T = 500     True_values                0.800000  0.600000  0.200000   
            Bias                      -0.328066  0.221879 -0.047461   
            std                        0.294164  0.123533  0.241364   
            RMSE                       0.440635  0.253950  0.245986   

                        nonstationary_no-constrain                      
                                           theta_1   theta_2   gamma_1  
Sample Size Values                                                      
T = 100     True_values                   0.300000  0.600000  0.200000  
            Bias                         -0.197951 -0.377481 -0.099000  
            std                           0.093396  0.158592  0.067455  
            RMSE                          0.218877  0.409443  0.119796  
T = 500     True_values                   0.300000  0.600000  0.200000  
            Bias                          0.933904  1.733066 -0.063656  
            std                           3.234536  5.296051  0.077893  
            RMSE                          3.366660  5.572402  0.100596

In [624]:
17*100/60

28.333333333333332

In [1356]:
a = [2,3,4,np.nan, 5,6]
np.nanstd(a)

1.4142135623730951